[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/diogoflim/Pesquisa-Operacional-III-A/blob/main/8_CPM_Impacto.ipynb)

# Gestão de Impacto em Projetos

In [ ]:
#Execute esse bloco caso esteja executando no Google Colab
!pip install -q pyomo
!apt-get install -y -qq coinor-cbc #instalação do solver gratuito

In [ ]:
import pyomo.environ as pe

### Exercício

Considere o problema de gerenciamento de projetos onde diversas atividades precisam ser executadas respeitando certos predecessores e limitações de tempo. 

Para cada uma das atividades "A" até "H", foram coletados: 

- o custo por período impactado;
- o número máximo de períodos que esta pode ser impactada para acelerar o seu término; 
- um tempo normal associado. 

O objetivo é minimizar o custo total de impacto, respeitando as restrições de tempo de predecessores, o tempo máximo que pode ser impactado e o tempo de entrega do projeto. 

Resolva com o Pyomo.


| Atividade | Precedências Diretas |
|-----------|----------------------|
| C         | A                    |
| D         | A                    |
| E         | B                    |
| F         | B                    |
| G         | C, E                 |
| H         | D, F                 |
| T         | G, H                 |


T indica o nó fictício de término do projeto. Para C iniciar, A deve ser concluída, e assim sucessivamente.


Parâmetros:



| Atividade | Custo do Impacto por Período | Tempo Normal | Redução Máxima de Tempo |
|-----------|------------------------------|--------------|-------------------------|
| A         | 6                            | 5            | 2                       |
| B         | 12                           | 3            | 1                       |
| C         | 4                            | 4            | 2                       |
| D         | 7                            | 6            | 3                       |
| E         | 10                           | 5            | 1                       |
| F         | 9                            | 7            | 3                       |
| G         | 6                            | 9            | 4                       |
| H         | 8                            | 8            | 2                       |

In [ ]:
# MODELO
M = pe.ConcreteModel() # Criando o modelo concreto para o nosso problema

#Atividades
A = M.atividades = pe.Set(initialize= ["A", "B", "C", "D", "E", "F", "G", "H"])
A_ = M.atividades_com_predecessores = pe.Set(initialize= ["C", "D", "E", "F", "G", "H", "T"])

# Parâmetros
M.c_impacto = pe.Param(M.atividades, initialize={"A": 6, "B": 12, "C": 4, "D": 7, "E": 10, "F": 9, "G": 6, "H": 8})
M.t_normal = pe.Param(M.atividades, initialize={"A": 5, "B": 3, "C": 4, "D": 6, "E": 5, "F": 7, "G": 9, "H": 8})
M.t_que_posso_impactar = pe.Param(M.atividades, initialize={"A": 2, "B": 1, "C": 2, "D": 3, "E": 1, "F": 3, "G": 4, "H": 2}) 

# VARIÁVEIS
M.x = pe.Var(A, domain= pe.PositiveReals) # Definindo a variável x
M.y = pe.Var(A_, domain= pe.PositiveReals) # Definindo a variável y

# FUNÇÃO OBJETIVO
z = M.z = pe.Objective(rule = lambda M: sum(M.x[i] * M.c_impacto[i] for i in M.atividades), sense = pe.minimize) # Definindo a função objetivo

# RESTRIÇÃO
M.R1 = pe.Constraint(A, expr= lambda M, i: M.x[i] <= M.t_que_posso_impactar[i]) # Definindo uma restrição de igualdade

M.R_CA = pe.Constraint(expr= M.y["C"] >= 0 + M.t_normal["A"] - M.x["A"])
M.R_DA = pe.Constraint(expr= M.y["D"] >= 0 + M.t_normal["A"] - M.x["A"])
M.R_EB = pe.Constraint(expr= M.y["E"] >= 0 + M.t_normal["B"] - M.x["B"])
M.R_FB = pe.Constraint(expr= M.y["F"] >= 0 + M.t_normal["B"] - M.x["B"])

M.R_GC = pe.Constraint(expr= M.y["G"] >= M.y["C"] + M.t_normal["C"] - M.x["C"])
M.R_GE = pe.Constraint(expr= M.y["G"] >= M.y["E"] + M.t_normal["E"] - M.x["E"])


M.R_HD = pe.Constraint(expr= M.y["H"] >= M.y["D"] + M.t_normal["D"] - M.x["D"])
M.R_HF = pe.Constraint(expr= M.y["H"] >= M.y["F"] + M.t_normal["F"] - M.x["F"])

M.R_TG = pe.Constraint(expr= M.y["T"] >= M.y["G"] + M.t_normal["G"] - M.x["G"])
M.R_TH = pe.Constraint(expr= M.y["T"] >= M.y["H"] + M.t_normal["H"] - M.x["H"])

M.Termino = pe.Constraint(expr = M.y["T"]<=16)

# SOLUCIONANDO O MODELO
pe.SolverFactory('cbc').solve(M)

# Imprime o resultado
print(f"z= {pe.value(M.z)}")
for k in M.x.keys(): 
    if pe.value(M.x[k]) > 0: 
        print (f"x_{k} = {pe.value(M.x[k])}")